In [1]:
!pip install tigramite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.6/309.6 kB 14.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from tigramite import plotting as tp
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats

In [3]:
def get_linear_equations(n_vars, max_lag):
    """Get linear equations for specified configuration with confounder"""
    if n_vars == 4:
        if max_lag == 2:
            return [
                "X4[t] = 0.25 * X1[t-2] + e4",
                "X3[t] = 0.35 * X4[t] + 0.3 * U[t] + e3",
                "X2[t] = 0.3 * X3[t-1] + e2",
                "X1[t] = 0.4 * X2[t] + 0.5 * U[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X4[t] = 0.25 * X1[t-2] + e4",
                "X3[t] = 0.35 * X4[t] + 0.2 * X2[t-3] + 0.3 * U[t] + e3",
                "X2[t] = 0.3 * X3[t-1] + e2",
                "X1[t] = 0.4 * X2[t] + 0.5 * U[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X4[t] = 0.25 * X1[t-4] + e4",
                "X3[t] = 0.35 * X4[t] + 0.2 * X2[t-3] + 0.3 * U[t] + e3",
                "X2[t] = 0.3 * X3[t-1] + e2",
                "X1[t] = 0.4 * X2[t] + 0.5 * U[t] + e1",
                "U[t] = eU"
            ]
    elif n_vars == 6:
        if max_lag == 2:
            return [
                "X6[t] = 0.85 * X5[t] + 0.4 * U[t] + e6",
                "X5[t] = e5",  # exogenous
                "X4[t] = 0.25 * X1[t-2] + 0.3 * X5[t-1] + e4",
                "X3[t] = 0.35 * X4[t] + 0.3 * U[t] + e3",
                "X2[t] = 0.3 * X3[t-1] + e2",
                "X1[t] = 0.4 * X2[t] + 0.5 * U[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X6[t] = 0.85 * X5[t] + 0.4 * U[t] + e6",
                "X5[t] = e5",  # exogenous
                "X4[t] = 0.25 * X1[t-2] + e4",
                "X3[t] = 0.35 * X4[t] + 0.2 * X2[t-3] + 0.3 * U[t] + e3",
                "X2[t] = 0.3 * X3[t-1] + e2",
                "X1[t] = 0.4 * X2[t] + 0.5 * U[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X6[t] = 0.85 * X5[t] + 0.4 * U[t] + e6",
                "X5[t] = e5",  # exogenous
                "X4[t] = 0.25 * X1[t-4] + e4",
                "X3[t] = 0.35 * X4[t] + 0.2 * X2[t-3] + 0.3 * U[t] + e3",
                "X2[t] = 0.3 * X3[t-1] + e2",
                "X1[t] = 0.4 * X2[t] + 0.5 * U[t] + e1",
                "U[t] = eU"
            ]
    elif n_vars == 8:
        if max_lag == 2:
            return [
                "X8[t] = 0.4 * X7[t] + 0.35 * U[t] + e8",
                "X7[t] = 0.35 * X6[t-1] + e7",
                "X6[t] = 0.45 * X5[t] + 0.4 * U[t] + e6",
                "X5[t] = e5",  # exogenous
                "X4[t] = 0.25 * X1[t-2] + 0.3 * X5[t-1] + e4",
                "X3[t] = 0.35 * X4[t] + 0.3 * U[t] + e3",
                "X2[t] = 0.3 * X3[t-1] + e2",
                "X1[t] = 0.4 * X2[t] + 0.5 * U[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X8[t] = 0.4 * X7[t] + 0.35 * U[t] + e8",
                "X7[t] = 0.35 * X6[t-1] + e7",
                "X6[t] = 0.45 * X5[t] + 0.4 * U[t] + e6",
                "X5[t] = e5",  # exogenous
                "X4[t] = 0.25 * X1[t-2] + e4",
                "X3[t] = 0.35 * X4[t] + 0.2 * X2[t-3] + 0.3 * U[t] + e3",
                "X2[t] = 0.3 * X3[t-1] + e2",
                "X1[t] = 0.4 * X2[t] + 0.5 * U[t] + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X8[t] = 0.4 * X7[t] + 0.35 * U[t] + e8",
                "X7[t] = 0.35 * X6[t-1] + e7",
                "X6[t] = 0.45 * X5[t] + 0.4 * U[t] + e6",
                "X5[t] = e5",  # exogenous
                "X4[t] = 0.25 * X1[t-4] + e4",
                "X3[t] = 0.35 * X4[t] + 0.2 * X2[t-3] + 0.3 * U[t] + e3",
                "X2[t] = 0.3 * X3[t-1] + e2",
                "X1[t] = 0.4 * X2[t] + 0.5 * U[t] + e1",
                "U[t] = eU"
            ]
    return []

class LinearTimeSeriesGenerator:
    def __init__(self, noise_type='gaussian', noise_scale=0.1, df=3, random_state=None):
        """
        Initialize time series generator with specified noise type

        Parameters:
        noise_type: str, 'gaussian' or 'student_t'
        noise_scale: float, scale parameter for noise distribution
        df: int, degrees of freedom for Student's t-distribution (used only if noise_type='student_t')
        random_state: int, random seed
        """
        self.noise_type = noise_type
        self.noise_scale = noise_scale
        self.df = df  # Degrees of freedom for t-distribution
        self.random_state = random_state
        if random_state is not None:
            np.random.seed(random_state)
            if noise_type == 'student_t':
                stats.t.random_state = np.random.RandomState(random_state)

    def generate_noise(self, size):
        """Generate noise from specified distribution"""
        if self.noise_type == 'gaussian':
            return np.random.normal(0, self.noise_scale, size=size)
        elif self.noise_type == 'student_t':
            return stats.t.rvs(df=self.df, loc=0, scale=self.noise_scale, size=size)
        else:
            raise ValueError(f"Unknown noise type: {self.noise_type}")

    def generate_linear_equations(self, t, X, U, n_vars, max_lag):
        """Execute linear equations with confounder"""
        noise = self.generate_noise(n_vars + 1)  # +1 for U
        equations = get_linear_equations(n_vars, max_lag)
        var_values = {}

        # Generate U first (confounder)
        U[t] = noise[-1]
        var_values['U'] = U[t]

        # Generate X5 if present (exogenous)
        if n_vars >= 6:
            X[t, 4] = noise[4]  # X5 is exogenous
            var_values['X5'] = X[t, 4]

        # Process other variables in the right order to respect dependencies
        # For most cases: X4 -> X3 -> X2 -> X1 -> X6 -> X7 -> X8

        # Process variables from highest to lowest index (except already handled)
        processed_indices = set()
        if n_vars >= 6:
            processed_indices.add(4)  # X5 already processed

        remaining_indices = list(range(n_vars))
        remaining_indices.reverse()  # Process from highest to lowest

        while remaining_indices:
            var_idx = remaining_indices[0]
            var_name = f"X{var_idx+1}"

            # Skip already processed
            if var_idx in processed_indices:
                remaining_indices.pop(0)
                continue

            # Find the equation for this variable
            eq = None
            for equation in equations:
                if equation.startswith(var_name):
                    eq = equation
                    break

            if not eq:
                # No equation found, move to next
                remaining_indices.pop(0)
                continue

            # Check if all dependencies are available
            left, right = eq.split('=')
            terms = [term.strip() for term in right.split('+')]

            all_deps_available = True
            for term in terms:
                if not (term.startswith('e') or 'U[t]' in term):
                    term_parts = term.split('*')
                    if len(term_parts) < 2:
                        continue

                    var_part = term_parts[1].strip()
                    if '[t]' in var_part:
                        dep_name = var_part.split('[')[0]
                        if dep_name not in var_values:
                            all_deps_available = False
                            break

            if not all_deps_available:
                # Move this to the end and try again later
                remaining_indices.pop(0)
                remaining_indices.append(var_idx)
                continue

            # Now compute the variable value
            value = 0.0

            for term in terms:
                if term.startswith('e'):  # Noise term
                    value += noise[var_idx]
                else:
                    # Parse coefficient and variable
                    parts = term.split('*')
                    coef = float(parts[0].strip())
                    var = parts[1].strip()

                    if 'U[t]' in var:
                        # Confounder term
                        value += coef * U[t]
                    else:
                        source_name = var.split('[')[0].strip()
                        time_idx = var.split('[')[1].split(']')[0].strip()
                        source_idx = int(source_name[1:]) - 1

                        if time_idx == 't':
                            # Current timestep dependency
                            if source_name in var_values:
                                # Use already computed value
                                value += coef * var_values[source_name]
                            else:
                                # Use value from X if not yet computed in this timestep
                                value += coef * X[t, source_idx]
                        else:
                            # Past timestep dependency
                            lag = int(time_idx.split('-')[1])
                            value += coef * X[t-lag, source_idx]

            # Store the computed value
            X[t, var_idx] = value
            var_values[var_name] = value
            processed_indices.add(var_idx)
            remaining_indices.pop(0)

    def generate_multivariate_ts(self, n_points, n_vars, max_lag):
        """Generate multivariate time series with regular sampling and confounder"""
        X = np.zeros((n_points, n_vars))
        U = np.zeros(n_points)  # Array for confounder U

        # Initialize first steps with noise
        for i in range(max_lag):
            X[i] = self.generate_noise(n_vars)
            U[i] = self.generate_noise(1)[0]

        # Generate time series
        for t in range(max_lag, n_points):
            self.generate_linear_equations(t, X, U, n_vars, max_lag)

        # Create DataFrame
        timestamps = np.arange(n_points)
        columns = [f'X{i+1}' for i in range(n_vars)]
        df = pd.DataFrame(X, columns=columns)
        df['U'] = U
        df['time'] = timestamps

        return df

def extract_linear_links(equations):
    """Extract all linear causal links from the equations"""
    links = {}

    for eq in equations:
        if '=' in eq:
            left, right = [side.strip() for side in eq.split('=')]
            target = left.split('[')[0]

            if target != 'U':  # Skip U's equation
                terms = [term.strip() for term in right.split('+')]
                for term in terms:
                    if '*' in term and ('X' in term or 'U' in term):
                        parts = term.split('*')
                        coeff = float(parts[0].strip())
                        var_part = parts[1].strip()
                        var = var_part.split('[')[0]
                        lag_part = var_part.split('[')[1].split(']')[0]

                        lag = 0 if lag_part == 't' else -int(lag_part.split('-')[1])
                        links[(var, lag, target)] = coeff

    return links

def save_dataset_and_graph(df, n_vars, max_lag, sample_size, noise_type, output_dir="output"):
    """Save dataset and create causal graph"""
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Save dataset
    filename = f'{output_dir}/linear_ts_with_confounder_n{sample_size}_vars{n_vars}_lag{max_lag}_{noise_type}.csv'
    df.to_csv(filename, index=False)

    # Get equations and extract links
    equations = get_linear_equations(n_vars, max_lag)
    true_links = extract_linear_links(equations)

    # Create matrices for tigramite plotting
    var_names = [f'X{i+1}' for i in range(n_vars)] + ['U']
    n_total_vars = n_vars + 1
    val_matrix = np.zeros((n_total_vars, n_total_vars, max_lag + 1))
    graph_matrix = np.zeros((n_total_vars, n_total_vars, max_lag + 1), dtype='bool')

    # Fill matrices based on true links
    for (source, lag, target), weight in true_links.items():
        if source == 'U':
            source_idx = n_vars
        else:
            source_idx = int(source[1:]) - 1

        target_idx = int(target[1:]) - 1
        lag_idx = abs(lag)

        # Add the link to the matrices
        val_matrix[source_idx, target_idx, lag_idx] = weight
        graph_matrix[source_idx, target_idx, lag_idx] = True

        # For contemporaneous links, make val_matrix symmetric
        if lag == 0:
            val_matrix[target_idx, source_idx, lag_idx] = weight

    # Plot and save causal graph
    plt.figure(figsize=(12, 12))
    tp.plot_time_series_graph(
        val_matrix=val_matrix,
        graph=graph_matrix,
        var_names=var_names,
        link_colorbar_label='Linear Effect Strength',
        node_size=0.05
    )
    plt.title(f'Linear Causal Graph with Confounder (n={sample_size}, vars={n_vars}, lag={max_lag}, {noise_type} noise)')
    plt.savefig(f'{output_dir}/linear_causal_graph_with_confounder_n{sample_size}_vars{n_vars}_lag{max_lag}_{noise_type}.png')
    plt.close()

    # Plot time series
    plt.figure(figsize=(15, 10))
    for col in df.columns[:-2]:  # Exclude U and time columns
        plt.plot(df['time'], df[col], label=col, alpha=0.7)
    plt.title(f'Linear Time Series with {noise_type.capitalize()} Errors (n={sample_size}, vars={n_vars}, lag={max_lag})')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{output_dir}/linear_ts_plot_with_confounder_n{sample_size}_vars{n_vars}_lag{max_lag}_{noise_type}.png')
    plt.close()

    # Save causal structure description
    with open(f'{output_dir}/linear_causal_structure_with_confounder_vars{n_vars}_lag{max_lag}_{noise_type}.txt', 'w') as f:
        f.write(f"True Linear Causal Structure with Confounder and {noise_type.capitalize()} Noise:\n")
        f.write("Format: (source, lag, target) => coefficient\n")
        f.write("\nEquations:\n")
        for eq in equations:
            f.write(f"{eq}\n")
        f.write("\nCausal Links:\n")
        for (source, lag, target), coef in true_links.items():
            f.write(f"({source}, {lag}, {target}) => {coef}\n")

def generate_all_combinations():
    """Generate datasets for all combinations"""
    n_vars_list = [4, 6, 8]
    max_lags = [2, 3, 4]
    sample_sizes = [500, 1000, 3000, 5000]
    noise_types = ['gaussian', 'student_t']

    for n in sample_sizes:
        for vars in n_vars_list:
            for lag in max_lags:
                for noise_type in noise_types:
                    print(f"\nGenerating dataset: n={n}, vars={vars}, lag={lag}, noise={noise_type}")

                    # Generate dataset with specified noise distribution
                    if noise_type == 'gaussian':
                        generator = LinearTimeSeriesGenerator(
                            noise_type='gaussian',
                            noise_scale=0.1,
                            random_state=42
                        )
                    else:  # student_t
                        generator = LinearTimeSeriesGenerator(
                            noise_type='student_t',
                            noise_scale=0.1,
                            df=3,
                            random_state=42
                        )

                    df = generator.generate_multivariate_ts(
                        n_points=n,
                        n_vars=vars,
                        max_lag=lag
                    )

                    # Save dataset and create visualizations
                    save_dataset_and_graph(df, vars, lag, n, noise_type)
                    print(f"Dataset and visualizations saved successfully")

# Function to verify generation is working properly
def test_generator(n_points=20, n_vars=4, max_lag=2, noise_type='gaussian'):
    """Function to test and verify the generator is working correctly"""
    generator = LinearTimeSeriesGenerator(
        noise_type=noise_type,
        noise_scale=0.1,
        random_state=42
    )

    df = generator.generate_multivariate_ts(
        n_points=n_points,
        n_vars=n_vars,
        max_lag=max_lag
    )

    print(f"Generated time series with shape: {df.shape}")
    print("\nSample of generated data:")
    print(df.head(10))

    # Check if there are any zeros (indicating potential issues)
    all_zeros = (df.iloc[:, :n_vars] == 0).all().any()
    if all_zeros:
        print("\nWARNING: Found columns with all zeros!")
    else:
        print("\nNo columns with all zeros detected.")

    # Basic statistical check
    print("\nBasic statistics:")
    print(df.iloc[:, :n_vars].describe())

    return df


In [4]:
if __name__ == "__main__":
    # First run a test to verify everything works
    print("Testing generator with small dataset...")
    test_df = test_generator(n_points=50, n_vars=4, max_lag=2, noise_type='gaussian')

    # If everything looks good, generate all combinations
    print("\nStarting generation of all combinations...")
    generate_all_combinations()

Testing generator with small dataset...
Generated time series with shape: (50, 6)

Sample of generated data:
         X1        X2        X3        X4         U  time
0  0.049671 -0.013826  0.064769  0.152303 -0.023415     0
1 -0.023414  0.157921  0.076743 -0.046947  0.054256     1
2 -0.142008 -0.023550 -0.090170 -0.178910 -0.172492     2
3 -0.178178 -0.128334 -0.044774 -0.096656 -0.141230     3
4  0.104942 -0.036010 -0.071871 -0.177977 -0.054438     4
5 -0.058157 -0.136661 -0.007794 -0.104608 -0.029169     5
6  0.054112  0.182890 -0.004511 -0.079536  0.082254     6
7 -0.104428  0.019533 -0.241636 -0.147358  0.019686     7
8 -0.022221 -0.055354 -0.061724 -0.016582 -0.147852     8
9 -0.185969 -0.064581  0.055710  0.008255 -0.176304     9

No columns with all zeros detected.

Basic statistics:
              X1         X2         X3         X4
count  50.000000  50.000000  50.000000  50.000000
mean    0.004605   0.002396  -0.002106  -0.007548
std     0.101130   0.096618   0.090392   0.0938

/usr/local/lib/python3.11/dist-packages/tigramite/plotting.py:3203: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = pyplot.figure(figsize=figsize)
<ipython-input-3-142a138f26a8>:334: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(15, 10))


Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=4, lag=3, noise=gaussian


<ipython-input-3-142a138f26a8>:321: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 12))


Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=4, lag=3, noise=student_t
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=4, lag=4, noise=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=4, lag=4, noise=student_t
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=2, noise=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=2, noise=student_t
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=3, noise=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=3, noise=student_t
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=4, noise=gaussian
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=6, lag=4, noise=student_t
Dataset and visualizations saved successfully

Gen

...

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

In [5]:
!zip -r /content/output_A1C.zip /content/output
from google.colab import files
files.download('/content/output_A1C.zip')

  adding: content/output/ (stored 0%)
  adding: content/output/linear_causal_graph_with_confounder_n5000_vars6_lag4_gaussian.png (deflated 4%)
  adding: content/output/linear_causal_graph_with_confounder_n500_vars4_lag2_gaussian.png (deflated 5%)
  adding: content/output/linear_ts_plot_with_confounder_n1000_vars8_lag3_gaussian.png (deflated 2%)
  adding: content/output/linear_ts_with_confounder_n1000_vars8_lag2_gaussian.csv (deflated 54%)
  adding: content/output/linear_ts_with_confounder_n5000_vars4_lag3_student_t.csv (deflated 54%)
  adding: content/output/linear_ts_with_confounder_n5000_vars4_lag2_gaussian.csv (deflated 54%)
  adding: content/output/linear_ts_with_confounder_n500_vars8_lag4_gaussian.csv (deflated 53%)
  adding: content/output/linear_ts_with_confounder_n500_vars8_lag3_gaussian.csv (deflated 53%)
  adding: content/output/linear_causal_graph_with_confounder_n3000_vars4_lag3_student_t.png (deflated 4%)
  adding: content/output/linear_causal_graph_with_confounder_n500_va

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>